In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import casadi as ca
from robot import *
from helper_fns import *

In [2]:
par = {'pos': [0, 0, 0.0],
       'stiff': [0, 0, 3e4],
        'rest':[0, 0, 0.7]}
attrs = yaml_load('config/attrs.yaml')
c = Contact('c/', par, ['rest'], attrs=attrs)
rob = LinearizedRobot('config/franka.urdf', subsys = [c], attrs = attrs, visc_fric = 100)

In [6]:
from observer import EKF

rob.build_step(0.002)
rob.build_linearized()
A, C, n, y = rob.linearized(np.zeros(rob.nx), np.zeros(7), rob.inv_mass_fn(np.zeros(7)))
ekf = EKF(rob, 0.002)
#%timeit ekf.step(0.01*np.ones(7), np.zeros(7), np.eye(7))
# 3x speedup from JITting the EKF step
for _ in range(100):
    ekf.step(0.01*np.ones(7), np.zeros(7))
print(ekf.get_ext_state())

{'q': array([[0.01      ],
       [0.00999999],
       [0.01      ],
       [0.01      ],
       [0.01      ],
       [0.01      ],
       [0.01      ]]), 'dq': array([[ 1.10828805e-07],
       [-4.44406060e-06],
       [ 1.12288126e-07],
       [ 1.93953032e-06],
       [ 9.27661821e-08],
       [-5.86774947e-08],
       [-5.66049631e-09]]), 'c/rest': array([[0.        ],
       [0.        ],
       [0.92604468]]), 'p': array([[0.09218158],
       [0.00267534],
       [0.92604468]]), 'R': array([[ 9.99750057e-01,  1.99986671e-02,  9.99384726e-03],
       [ 1.99956680e-02, -9.99799987e-01,  3.99926852e-04],
       [ 9.99984636e-03, -1.99993241e-04, -9.99949980e-01]]), 'dx': array([[-3.18131577e-06],
       [-9.16125791e-09],
       [ 4.17260814e-07]]), 'c/F': array([[ 0.0000000e+00],
       [ 0.0000000e+00],
       [-2.5809288e-05]]), 'c/disp': array([[9.21815812e-02],
       [2.67533627e-03],
       [8.60309601e-10]]), 'c/n': array([[0.],
       [0.],
       [1.]]), 'c/x': array([[0.0

In [42]:
from impedance_controller import ImpedanceController
from robot import Robot
imp = ImpedanceController(input_vars = [])
rob = Robot('config/franka.urdf', ctrl = imp, visc_fric=50)
rob.build_step(0.05)
# test that it converges as expected 
q = np.zeros(7)
dq = np.zeros(7)
state = {'q':q, 'dq':dq, 'M_inv':rob.inv_mass_fn(q), 
         'imp_stiff':[400, 400, 200], 'imp_rest':[0.0,0.0,0.5]}
#%timeit step(**state)

print(rob.step)
for _ in range(300):
    res = rob.step.call(state)
    state['q'] = res['q']
    state['dq'] = res['dq']
    #print(res['dq'])
    #state['M_inv'] = rob.inv_mass_fn(state['q'])
    res_dict = rob.get_ext_state(state)
    print(res_dict['p'][2])

#print(rob.inv_mass_fn(np.zeros(7)))


step:(q[7],dq[7],M_inv[7x7],imp_stiff[3],imp_rest[3])->(q[7],dq[7],cost) SXFunction
[0.9256293]
[0.92536755]
[0.92511688]
[0.92472011]
[0.92407367]
[0.92314529]
[0.92196256]
[0.92059185]
[0.91911622]
[0.91761718]
[0.91616247]
[0.91479993]
[0.91355639]
[0.91244009]
[0.91144489]
[0.91055509]
[0.90974986]
[0.90900691]
[0.90830504]
[0.90762574]
[0.90695398]
[0.90627832]
[0.90559064]
[0.90488568]
[0.90416042]
[0.90341357]
[0.90264508]
[0.90185574]
[0.90104689]
[0.90022021]
[0.89937753]
[0.89852076]
[0.89765178]
[0.89677237]
[0.89588418]
[0.8949887]
[0.89408721]
[0.89318075]
[0.89227016]
[0.89135602]
[0.8904387]
[0.88951838]
[0.88859504]
[0.88766848]
[0.88673841]
[0.8858044]
[0.88486597]
[0.88392257]
[0.88297362]
[0.88201853]
[0.88105673]
[0.88008765]
[0.87911073]
[0.87812546]
[0.87713137]
[0.876128]
[0.87511493]
[0.8740918]
[0.87305824]
[0.87201394]
[0.87095859]
[0.86989193]
[0.86881369]
[0.86772362]
[0.86662151]
[0.86550714]
[0.86438028]
[0.86324075]
[0.86208835]
[0.86092288]
[0.85974416]
